In [11]:
import json
import requests
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.nodes import TfidfRetriever
from haystack import Document

In [5]:
with open('data/squad/dev.json', 'r') as file:
    squad = json.load(file)

content = [sample['context'] for sample in squad]

In [7]:
print(f'Content lengh before {len(content)}')
content = list(set(content))
print(f'Content lengh after {len(content)}')


Content lengh before 11873
Content lengh after 1204


In [12]:
squad_docs = [Document(content=sample, content_type='text') for sample in content]
squad_docs[0]

<Document: {'content': "During 2003–04, the gross value of Victorian agricultural production increased by 17% to $8.7 billion. This represented 24% of national agricultural production total gross value. As of 2004, an estimated 32,463 farms occupied around 136,000 square kilometres (52,500 sq mi) of Victorian land. This comprises more than 60% of the state's total land surface. Victorian farms range from small horticultural outfits to large-scale livestock and grain productions. A quarter of farmland is used to grow consumable crops.", 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': 'ad766425af006b2689b748dea7197b56'}>

In [13]:

document_store = ElasticsearchDocumentStore(
    host='localhost', 
    port=9200,
    index='squad_docs'
)

/Users/eliranboraks/opt/anaconda3/envs/haystack/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/Users/eliranboraks/opt/anaconda3/envs/haystack/lib/python3.8/site-packages/haystack/document_stores/elasticsearch.py:440: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  indices = self.client.indices.get(index_name, headers=headers)


In [9]:
res = requests.get('http://localhost:9200/_cluster/health')

In [10]:
res.json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 5,
 'active_shards': 5,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 71.42857142857143}

In [14]:
document_store.write_documents(squad_docs)

In [27]:
retriever = TfidfRetriever(document_store)

/Users/eliranboraks/opt/anaconda3/envs/nlp2/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
11/09/2022 08:54:08 - INFO - elasticsearch -   POST http://localhost:9200/squad_docs/_search?scroll=1d&size=10000 [status:200 request:0.162s]
11/09/2022 08:54:08 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.032s]
11/09/2022 08:54:08 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.003s]
11/09/2022 08:54:08 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.002s]
11/09/2022 08:54:08 - INFO - haystack.retriever.sparse -

In [30]:
query = squad[6]['question']

In [31]:
retriever.retrieve(query)

[{'text': 'After this, Huguenots (with estimates ranging from 200,000 to 1,000,000) fled to surrounding Protestant countries: England, the Netherlands, Switzerland, Norway, Denmark, and Prussia — whose Calvinist Great Elector Frederick William welcomed them to help rebuild his war-ravaged and underpopulated country. Following this exodus, Huguenots remained in large numbers in only one region of France: the rugged Cévennes region in the south. In the early 18th century, a regional group known as the Camisards who were Huguenots rioted against the Catholic Church in the region, burning churches and killing clergy. It took French troops years to hunt down and destroy all the bands of Camisards, between 1702 and 1709.', 'id': 'e0bab9de-df57-45fe-9fc5-85a02dea55c0', 'score': None, 'probability': None, 'question': None, 'meta': {}, 'embedding': None},
 {'text': 'After this, Huguenots (with estimates ranging from 200,000 to 1,000,000) fled to surrounding Protestant countries: England, the Ne